### importing the libraries

In [1]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [32]:
#loading the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

True

In [33]:
api_key = os.getenv("PINE_KEY")

In [7]:
import os
from pinecone import Pinecone, ServerlessSpec


index_name = "hybrid-search-pinecone"

#initizing the pinecone clinet
pc = Pinecone(api_key=api_key)

try:
    # Retrieve the list of existing indexes
    existing_indexes = pc.list_indexes()
    print(f"Existing indexes: {existing_indexes}")

    if index_name not in existing_indexes:
        # Create the index if it doesn't exist
        pc.create_index(
            name=index_name,
            dimension=384,  # Dimension of your dense model
            metric='dotproduct',  # Metric for similarity search
            spec=ServerlessSpec(cloud='aws', region='us-east-1')
        )
        print(f"Index '{index_name}' created successfully.")
    else:
        print(f"Index '{index_name}' already exists.")

except Exception as e:
    if e.status == 409:
        # Handle the case where the index already exists
        print(f"Index '{index_name}' already exists. Proceeding to use it.")
    else:
        # Handle other potential exceptions
        print(f"An error occurred while creating the index: {e}")

Existing indexes: {'indexes': [{'deletion_protection': 'disabled',
              'dimension': 384,
              'host': 'hybrid-search-pinecone-nqxn8hs.svc.aped-4627-b74a.pinecone.io',
              'metric': 'dotproduct',
              'name': 'hybrid-search-pinecone',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}
Index 'hybrid-search-pinecone' already exists. Proceeding to use it.


In [8]:
index = pc.Index(index_name)
index

True

In [9]:


from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")


In [10]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default() #defining the default encoder i.e .default()
bm25_encoder

In [12]:
#defining the test sentences
sentences = [
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans",
]

In [13]:
#resolving the nltk error 
import nltk

# Download the punkt tokenizer model
nltk.download('punkt_tab')




[nltk_data] Downloading package punkt_tab to /Users/vivek/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [14]:
#tfidf values  on these sentences
bm25_encoder.fit(sentences)


100%|██████████| 3/3 [00:00<00:00, 259.99it/s]


In [15]:
#store the values as json files
bm25_encoder.dump("bm25_values.json")

#loading the encoder object
bm25_loaded = BM25Encoder().load("bm25_values.json")

In [20]:
#creating a retriver object
retriver = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder, index=index)

In [21]:
retriver

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x177838d90>, index=<pinecone.data.index.Index object at 0x10e0f8710>)

In [23]:
retriver.add_texts([
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans",
])

100%|██████████| 1/1 [00:01<00:00,  1.79s/it]


In [27]:
retriver.invoke("What city did i visit in 2022.?")

[Document(metadata={'score': 0.391231596}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.347800106}, page_content='In 2023, I visited Paris'),
 Document(metadata={'score': 0.34340322}, page_content='In 2021, I visited New Orleans')]